In [1]:
import warnings
warnings.filterwarnings("ignore")

import gc  
import os  
import time  
import joblib  

import numpy as np  
import numba as nb
import pandas as pd
import polars as pl
import lightgbm as lgb  

from numba import njit, prange
from itertools import combinations  
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import KFold, TimeSeriesSplit  

device = "gpu"

is_offline = False 
is_train = True  
is_infer = True 
runtime = False
split_day = 435  

# Data Loading






In [2]:
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
df = df.dropna(subset=["target"])
df.reset_index(drop=True, inplace=True)
gc.collect()
df_shape = df.shape

# Feature Generation Functions 

In [3]:
@nb.jit(nopython=False, parallel=True)
def reduce_mem_usage(df, verbose=0):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
               
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    return df


@nb.jit(nopython=False, parallel=True)
def get_stock_weight(sizes, waps):
    matched_volume = sizes * waps
    total_vol = matched_volume.sum()
    weights = matched_volume / total_vol
    return weights

def create_weights(df):
    if runtime:
        df['weights'] = get_stock_weight(df['matched_size'], df['wap'])   
    else:
        df['weights'] = df.groupby('time_id').apply(lambda x: get_stock_weight(x['matched_size'], x['wap'])).reset_index(level=0, drop=True)
    return df

    
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features



@nb.jit(nopython=False, parallel=True)
def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)
    return features



@nb.jit(nopython=False, parallel=True)
def calculate_weighted_wap(df):
    df["stock_weights"] = df["stock_id"].map(weights)
    df["weighted_wap"] = df["stock_weights"] * df["wap"]
    df["wap_momentum"] = df.groupby("stock_id")["weighted_wap"].pct_change(periods=6)
    # df = create_weights(df)
    # df["weighted_wap_2"] = df["weights"] * df["wap"]
    # df["wap_momentum_2"] = df.groupby("stock_id")["weighted_wap_2"].pct_change(periods=6)
    return df



@nb.jit(nopython=False, parallel=True)
def compute_combination_features(df, prices):
    for c in combinations(prices, 2):
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]} - {c[1]}) / ({c[0]} + {c[1]})')
    return df



@nb.jit(nopython=False, parallel=True)
def calculate_all_prices_and_sizes_features(df, prices, sizes):
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
    return df



@nb.jit(nopython=False, parallel=True)
def calculate_shift_ret_features(df):
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 3, 5, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
    return df



@nb.jit(nopython=False, parallel=True)
def calculate_diff_features(df):
    # Calculate diff features for specific columns
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'weighted_wap', 'price_spread']:
        for window in [1, 3, 5, 10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
    return df



@nb.jit(nopython=False, parallel=True)
def calculate_change_diff_features(df):
    for window in [3, 5, 10]:
        df[f'price_change_diff_{window}'] = df[f'bid_price_diff_{window}'] - df[f'ask_price_diff_{window}']
        df[f'size_change_diff_{window}'] = df[f'bid_size_diff_{window}'] - df[f'ask_size_diff_{window}']
    return df



@nb.jit(nopython=False, parallel=True)
def calculate_rolling_expressions(df):
    # Convert from pandas to Polars
    pl_df = pl.from_pandas(df)

    # Define the windows and columns for which you want to calculate the rolling statistics
    windows = [3, 5, 10]
    columns = ['ask_price', 'bid_price', 'ask_size', 'bid_size']

    # prepare the operations for each column and window
    group = ["stock_id"]
    expressions = []

    # Loop over each window and column to create the rolling mean and std expressions
    for window in windows:
        for col in columns:
            rolling_mean_expr = (
                pl.col(f"{col}_diff_{window}")
                .rolling_mean(window)
                .over(group)
                .alias(f'rolling_diff_{col}_{window}')
            )
            
            rolling_std_expr = (
                pl.col(f"{col}_diff_{window}")
                .rolling_std(window)
                .over(group)
                .alias(f'rolling_std_diff_{col}_{window}')
            )
            
            # Added
            rolling_skew_expr = (
                pl.col(f"{col}_diff_{window}")
                .rolling_skew(window)
                .over(group)
                .alias(f'rolling_skew_diff_{col}_{window}')
            )
            
            expressions.append(rolling_mean_expr)
            expressions.append(rolling_std_expr)
            expressions.append(rolling_skew_expr)
            
    # Run the operations using Polars' lazy API
    lazy_df = pl_df.lazy().with_columns(expressions)
    
    # Execute the lazy expressions and overwrite the pl_df variable
    pl_df = lazy_df.collect()
    
    # Convert back to pandas if necessary
    df = pl_df.to_pandas()
    return df


@nb.jit(nopython=False, parallel=True)
def calculate_bollinger(df):
    # Bollinger Bands 
    periods = [5, 10]
    for col in ['ask_price', 'bid_price', 'wap']:
        grouped = df.groupby(["stock_id", "date_id"])[col]
        for p in periods: 
            grouped_std = grouped.rolling(window=p).std().reset_index(drop=True)            
            df[f"{col}_bollinger_upper_{p}"] = df[f"{col}"] + 2 * grouped_std
            df[f"{col}_bollinger_lower_{p}"] = df[f"{col}"] - 2 * grouped_std
    return df


@nb.jit(nopython=False, parallel=True)
def calculate_norm_features(df):   
    grouped = df.groupby(["stock_id", "date_id"])
    for col in ['ask_price', 'bid_price', 'wap', 'reference_price']:
        cummax = grouped[col].cummax() 
        cummin = grouped[col].cummin() 
        df[f"{col}_min_max_norm"] = (df[f'{col}'] - cummin) / (cummax - cummin)
    return df


def imbalance_features(df):
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")

    df = calculate_weighted_wap(df)
   
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    
    df = calculate_diff_features(df)
    df = calculate_rolling_expressions(df)
    df = calculate_bollinger(df)
    df = calculate_norm_features(df)
    
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    
    df['spread_depth_ratio'] = (df['ask_price'] - df['bid_price']) / (df['bid_size'] + df['ask_size'])
    df['mid_price_movement'] = df['mid_price'].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    
    df['micro_price'] = ((df['bid_price'] * df['ask_size']) + (df['ask_price'] * df['bid_size'])) / (df['bid_size'] + df['ask_size'])
    df['relative_spread'] = (df['ask_price'] - df['bid_price']) / df['wap']
        
    df = calculate_shift_ret_features(df)
    df = calculate_change_diff_features(df)
    
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
        
    df = compute_combination_features(df, prices)
    df = calculate_all_prices_and_sizes_features(df, prices, sizes)
    
    df['mid_price*volume'] = df['mid_price_movement'] * df['volume']
    df['harmonic_imbalance'] = df.eval('2 / ((1 / bid_size) + (1 / ask_size))')
    
    df.replace([np.inf, -np.inf], 0, inplace=True)
    gc.collect()
    
    return df


@nb.jit(nopython=False, parallel=True)
def other_features(df):
    df["dow"] = df["date_id"] % 5  # Day of the week
    df["dom"] = df["date_id"] % 20
    df["seconds"] = df["seconds_in_bucket"] % 60  
    df["minute"] = df["seconds_in_bucket"] // 60  
    df['time_to_market_close'] = 540 - df['seconds_in_bucket']
    
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

def generate_all_features(df):
    # Select relevant columns for feature generation
    cols = [c for c in df.columns if c not in ["row_id", "target"]]
    df = df[cols]
    
    # Generate imbalance features
    df = imbalance_features(df)
    df = other_features(df)
    
    feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id"]]
    
    gc.collect()
    return df[feature_name]


weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]

weights = {int(k):v for k,v in enumerate(weights)}

## Data Splitting

In [4]:
if is_offline:
    df_train = df[df["date_id"] <= split_day]
    df_valid = df[df["date_id"] > split_day]
    print("Offline mode")
    print(f"train : {df_train.shape}, valid : {df_valid.shape}")
else:
    df_train = df
    print("Online mode")

Online mode


In [5]:
if is_train:
    global_stock_id_feats = {
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
    }
    
    if is_offline:
        df_train_feats = generate_all_features(df_train)
        print("Build Train Feats Finished.")
        
        df_valid_feats = generate_all_features(df_valid)
        print("Build Valid Feats Finished.")
        
        df_valid_feats = reduce_mem_usage(df_valid_feats)
    
    else:
        df_train_feats = generate_all_features(df_train)
        print("Build Online Train Feats Finished.")
    
    gc.collect()
    
    # Reduce the memory usage of the dataset
    df_train_feats = reduce_mem_usage(df_train_feats)

Build Online Train Feats Finished.


# **Model Training**

In [6]:
log_evaluation = 2
stopping_rounds = 100

gap = 5 # 5
purge = 2 # 2

if device == "gpu":
    print("Using GPU params\n")
    num_folds = 5
    fold_size = 480 // num_folds
    
    lgb_params = {
        "objective": "mae",
        "n_estimators": 6000,
        #"num_leaves": 256,
        "num_leaves": 252,
        "subsample": 0.62,
        "colsample_bytree": 0.73,
        "learning_rate": 0.01,
        "max_depth": 11,
        "n_jobs": 4,
        "importance_type": "gain",
        "reg_alpha": 0.20,
        "reg_lambda": 3.28,
        #"path_smooth": 0.2,         # Added
        "bagging_freq": 5,           # Added
        "min_gain_to_split": 0.025,
        "device": "gpu",
        "verbosity": 0
    }

else:
    
    print("Using CPU params\n")
    num_folds = 4
    fold_size = 480 // num_folds
    
    lgb_params = {
        "objective": "mae",
        "n_estimators": 3400,
        "num_leaves": 146,
        "subsample": 0.620,
        "colsample_bytree": 0.72,
        "learning_rate": 0.01,
        "max_depth": 10,
        "data_sample_strategy": "goss",
        "other_rate": 0.15,
        "importance_type": "gain",
        "reg_alpha": 0.16,
        "reg_lambda": 3.00,
        #"path_smooth": 0.05,        # Added
        "bagging_freq": 6,           # Added
        "min_gain_to_split": 0.025,
        "device": "cpu",
        "verbosity": 0
    }
    

feature_columns = list(df_train_feats.columns)
print(f"Features = {len(feature_columns)}")

date_ids = df_train['date_id'].values

model_save_path = 'models_for_competion_mae_3' 
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

gc.collect()

models = []
for i in range(num_folds):
    print(f"Fold {i+1} Model Training")
    
    start = i * fold_size
    end = start + fold_size
    if i < num_folds - 1:  # No need to purge after the last fold
        purged_start = end - purge
        purged_end = end + gap + purge
        train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
    else:
        train_indices = (date_ids >= start) & (date_ids < end)

    test_indices = (date_ids >= end) & (date_ids < end + fold_size)

    df_fold_train = df_train_feats[train_indices]
    df_fold_train_target = df_train['target'][train_indices]
    df_fold_valid = df_train_feats[test_indices]
    df_fold_valid_target = df_train['target'][test_indices]
    
    gc.collect()
    
    # Train a LightGBM model for the current fold
    lgb_model = lgb.LGBMRegressor(**lgb_params)
    lgb_model.fit(
        df_fold_train[feature_columns],
        df_fold_train_target,
        eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)],
        callbacks=[
            lgb.callback.early_stopping(stopping_rounds=stopping_rounds),
            lgb.callback.log_evaluation(period=log_evaluation),
        ],
    )

    models.append(lgb_model)
    
    # Save the model to a file
    model_filename = os.path.join(model_save_path, f'model_fold_{i+1}.txt')
    lgb_model.booster_.save_model(model_filename)
    print(f"Model for fold {i+1} saved to {model_filename}")

    # Free up memory by deleting fold specific variables
    del df_fold_train, df_fold_train_target, df_fold_valid, df_fold_valid_target
    
    gc.collect()
    
    print("\n\n")
    
    
final_model_params = lgb_params.copy()

# Train the final model on the entire dataset
num_model = 1

for i in range(num_model):
    final_model = lgb.LGBMRegressor(**final_model_params)
    final_model.fit(
        df_train_feats[feature_columns],
        df_train['target'],
        callbacks=[
            lgb.callback.log_evaluation(period=log_evaluation),
        ],
    )
    
    # Append the final model to the list of models
    models.append(final_model)

Using GPU params

Features = 189
Fold 1 Model Training
[LightGBM] [Warning] min_gain_to_split is set=0.025, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.025
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_gain_to_split is set=0.025, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.025
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_gain_to_split is set=0.025, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.025
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 7.23748
[4]	valid_0's l1: 7.23191
[6]	valid_0's l1: 7.22646
[8]	valid_0's l1: 7.22128
[10]	valid_0's l1: 7.21607
[12]	valid_0's l1: 7.21108
[14]	valid_0's l1

# **Submission**

In [7]:
@nb.jit(nopython=False, parallel=True)
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices) / np.sum(std_error)
    out = prices - std_error * step
    return out


def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return w


@nb.jit(nopython=False, parallel=True)
def get_final_predictions(predictions, y_min, y_max):
    mean = np.mean(predictions)
    median = np.median(predictions)
    final_predictions = predictions - (0.925 * mean + 0.075 * median)
    final_predictions = np.clip(final_predictions, y_min, y_max)
    return final_predictions


if is_infer:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    
    runtime = True
    
    counter = 0
    y_min, y_max = -64, 64
    
    qps, predictions = [], []
    
    cache = pd.DataFrame()

    # Weights for each fold model
    lgb_model_weights = weighted_average(models)
    
    for (test, revealed_targets, sample_prediction) in iter_test:
        now_time = time.time()

        cache = pd.concat([cache, test], ignore_index=True, axis=0)
        if counter > 0:
            cache = cache.groupby(['stock_id']).tail(21).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
        
        feat = generate_all_features(cache)[-len(test):]

        # Generate predictions for each model and calculate the weighted average
        lgb_predictions = np.zeros(len(test))
        for model, weight in zip(models, lgb_model_weights):
            lgb_predictions += weight * model.predict(feat[feature_columns])
        
        predictions = lgb_predictions
        final_predictions = predictions - np.mean(predictions)
        clipped_predictions = np.clip(final_predictions, y_min, y_max)
        
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)
        
        counter += 1
        
        qps.append(time.time() - now_time)
        if counter % 10 == 0:
            print(counter, 'qps:', np.mean(qps))

    time_cost = 1.146 * np.mean(qps)
    print(f"The code will take approximately {np.round(time_cost, 4)} hours to reason about")

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
10 qps: 1.4927611112594605
20 qps: 1.4409387946128844
30 qps: 1.4483315070470175
40 qps: 1.461018568277359
50 qps: 1.4618146276474
60 qps: 1.4734557231267293
70 qps: 1.4858218465532576
80 qps: 1.4934225767850875
90 qps: 1.4922581990559896
100 qps: 1.488925919532776
110 qps: 1.4868779984387483
120 qps: 1.4963797728220622
130 qps: 1.499857759475708
140 qps: 1.4960429549217225
150 qps: 1.4945502948760987
160 qps: 1.492723675072193
The code will take approximately 1.7128 hours to reason about
